<a href="https://colab.research.google.com/github/rainerhosch/machine-learning-research/blob/main/Jaringan-Syaraf/Penjadwalan_Kuliah_JST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder


In [2]:
# 1. Parameter Dataset
num_courses = 100  # Jumlah mata kuliah
num_lecturers = 20  # Jumlah dosen
num_time_slots = 10  # Jumlah slot waktu
num_rooms = 5  # Jumlah ruangan

# 2. Generate Data
np.random.seed(42)  # Untuk hasil yang konsisten
data = {
    "Course_Code": [f"C{str(i).zfill(3)}" for i in range(num_courses)],
    "Lecturer_ID": np.random.choice(range(1, num_lecturers + 1), num_courses),
    "Time_Slot": np.random.choice(range(1, num_time_slots + 1), num_courses),
    "Room_ID": np.random.choice(range(1, num_rooms + 1), num_courses),
}

# 3. Buat DataFrame
df = pd.DataFrame(data)

# 4. Simpan ke CSV
df.to_csv('schedule_data.csv', index=False)

print("Dataset Dummy berhasil dibuat!")
print(df.head())


Dataset Dummy berhasil dibuat!
  Course_Code  Lecturer_ID  Time_Slot  Room_ID
0        C000            7          3        4
1        C001           20          8        3
2        C002           15          6        4
3        C003           11          3        3
4        C004            8          1        2


In [39]:


# Load data
dataset = pd.read_csv("/content/data_jadwal_kuliah.csv")

# Encode categorical data
encoder_waktu = LabelEncoder()
encoder_ruangan = LabelEncoder()
dataset['Slot_Waktu'] = encoder_waktu.fit_transform(dataset['Slot_Waktu'])
dataset['Ruangan'] = encoder_ruangan.fit_transform(dataset['Ruangan'])

# Define features (X) and target (y)
X = dataset[['Kode_Mk', 'Id_Dosen']].copy()
X['Kode_Mk'] = LabelEncoder().fit_transform(X['Kode_Mk'])
y = dataset[['Slot_Hari', 'Slot_Waktu', 'Ruangan']]

# Split and scale data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define ANN model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='linear')
])

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Save the model
model.save("jadwal_model.h5")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 71ms/step - loss: 202.9933 - mae: 10.8320 - val_loss: 168.5055 - val_mae: 9.5741
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 181.2672 - mae: 9.9415 - val_loss: 126.7921 - val_mae: 8.0308
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 121.7309 - mae: 7.7526 - val_loss: 74.4467 - val_mae: 6.4250
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 72.9282 - mae: 6.1734 - val_loss: 67.4529 - val_mae: 6.0958
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 68.0709 - mae: 6.0499 - val_loss: 61.4467 - val_mae: 5.9367
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 63.4750 - mae: 5.8082 - val_loss: 60.5999 - val_mae: 5.9169
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 62.9979 - mae: 5.8558 - val_loss: 60.1181 - val_mae: 5.8743
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 63.8016 - mae: 5.8630 - val_loss: 59.3226 - val_mae: 5.8376
Epoch 9/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 59

Test Loss: 53.228816986083984, Test MAE: 5.581546783447266


In [40]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("jadwal_model.h5")



In [41]:
import pandas as pd

# Load new data
new_data = pd.DataFrame({
    'Kode_Mk': ['TT19111', 'DK19131'],  # contoh data
    'Id_Dosen': [147, 42]
})

# Encode and scale new data
new_data['Kode_Mk'] = LabelEncoder().fit_transform(new_data['Kode_Mk'])  # Gunakan encoder lama
new_data_scaled = scaler.transform(new_data)


In [42]:
# Predict schedule for new data
predictions = model.predict(new_data_scaled)

# Decode predictions
decoded_predictions = {
    'Slot_Hari': predictions[:, 0].round().astype(int),
    'Slot_Waktu': encoder_waktu.inverse_transform(predictions[:, 1].round().astype(int)),
    'Ruangan': encoder_ruangan.inverse_transform(predictions[:, 2].round().astype(int))
}

result = pd.DataFrame(decoded_predictions)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
   Slot_Hari   Slot_Waktu Ruangan
0          3  12:30-14:10      17
1          4  18:00-19:40      32
